In [2]:
!pip install pulp
import pulp

model = pulp.LpProblem("Forklift_Optimization", pulp.LpMinimize)
C1, C2, C3 = 150, 120, 25
num_shipments = 23
num_docks = 6
num_hours = 4

time_windows = {
    1: [1, 2], 17: [1, 2], 21: [1, 2], 20: [1, 2],
    2: [1, 2, 3, 4], 5: [1, 2, 3, 4], 9: [1, 2, 3, 4], 14: [1, 2, 3, 4],
    3: [1, 2], 8: [1, 2], 12: [1, 2], 16: [1, 2], 19: [1, 2],
    4: [2, 3], 6: [2, 3], 7: [2, 3], 18: [2, 3],
    10: [1, 2, 3, 4], 11: [1, 2, 3, 4], 22: [1, 2, 3, 4], 23: [1, 2, 3, 4],
    13: [3, 4], 15: [3, 4]
}

unloading_times = {
    1: 1, 17: 0.5, 21: 1.5, 20: 1,
    2: 2, 5: 1, 9: 1, 14: 3,
    3: 2.5, 8: 1, 12: 2, 16: 1.5, 19: 2,
    4: 2, 6: 1.5, 7: 2, 18: 3,
    10: 3, 11: 1, 22: 1, 23: 1.5,
    13: 2, 15: 2
}

x1 = pulp.LpVariable("x1", lowBound=0, cat="Integer")  # Type 1 forklifts
x2 = pulp.LpVariable("x2", lowBound=0, cat="Integer")  # Type 2 forklifts
x3 = pulp.LpVariable("x3", lowBound=0, cat="Integer")  # Type 3 forklifts

t = pulp.LpVariable.dicts("t", [(i, j, k) for i in range(1, num_shipments + 2)
                                for j in range(1, num_docks + 1)
                                for k in range(1, num_hours + 1)], cat="Binary")
T = pulp.LpVariable.dicts("T", range(1, num_shipments + 2), lowBound=0)

model += (
    C1 * x1 + C2 * x2 + C3 * x3 + pulp.lpSum(T[i] for i in range(1, num_shipments + 1)),
    "Total Cost"
)

# Constraints
# Ensure only one shipment is assigned per dock per hour
for j in range(1, num_docks + 1):
    for k in range(1, num_hours + 1):
        model += (
            pulp.lpSum(t[i, j, k] for i in range(1, num_shipments + 1)) <= 1,
            f"One_Shipment_Per_Dock_Per_Hour_Dock{j}_Hour{k}"
        )

# Each shipment can be assigned to one dock within its time window
for i in time_windows.keys():
    model += (
        pulp.lpSum(t[i, j, k] for j in range(1, num_docks + 1) for k in time_windows[i]) == 1,
        f"Single_Assignment_Per_Shipment_{i}"
    )

# Type-based unloading time constraints
for i in time_windows.keys():
    model += T[i] == (
        0.25 * unloading_times[i] * x1 + 0.5 * unloading_times[i] * x2 + 0.25 * unloading_times[i] * x3
    ), f"Unloading_Time_For_Shipment_{i}"

model += pulp.lpSum(unloading_times[i] for i in time_windows.keys()) <= 6 * x1 + 4 * x2 + 6 * x3

model.solve()

print("Status:", pulp.LpStatus[model.status])
print("Objective Value:", pulp.value(model.objective))
print("Type 1 Forklifts:", x1.varValue)
print("Type 2 Forklifts:", x2.varValue)
print("Type 3 Forklifts:", x3.varValue)
for i in time_windows.keys():
    print(f"T[{i}] (Unloading Time): {T[i].varValue}")
for i in time_windows.keys():
    for j in range(1, num_docks + 1):
        for k in range(1, num_hours + 1):
            if t[i, j, k].varValue is not None and t[i, j, k].varValue > 0:
                print(f"Shipment {i} assigned to Dock {j} at Hour {k}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 51.3 MB/s eta 0:00:00
Status: Optimal
Objective Value: 243.25
Type 1 Forklifts: 0.0
Type 2 Forklifts: 0.0
Type 3 Forklifts: 7.0
T[1] (Unloading Time): 1.75
T[17] (Unloading Time): 0.875
T[21] (Unloading Time): 2.625
T[20] (Unloading Time): 1.75
T[2] (Unloading Time): 3.5
T[5] (Unloading Time): 1.75
T[9] (Unloading Time): 1.75
T[14] (Unloading Time): 5.25
T[3] (Unloading Time): 4.375
T[8] (Unloading Time): 1.75
T[12] (Unloading Time): 3.5
T[16] (Unloading Time): 2.625
T[19] (Unloading Time): 3.5
T[4] (Unloading Time): 3.5
T[6] (Unloading Time): 2.625
T[7] (Unloading Time): 3.5
T[18] (Unloading Time): 5.25
T[10] (Unloading Time): 5.25
T[11] (Unloading Time): 1.75
T[22] (Unloading Time): 1.75
T[23] (Unloading Time): 2.625
T[13] (Unloading Time): 3.5
T[15] (Unloading Time): 3.5
Shipment 1 assigned to Dock 3 at Hour 2
Shipment 17 assigned to Dock 4 at Hour 1
Shipment 21 assigned to Dock 3 at Hour 1
Shipment 20 assigned to Dock 1 at Ho